# Scratch pad to write ResNext network architecture for TinyImageNet project

### various imports

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
from keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Activation, Dropout
from keras.layers import concatenate, AveragePooling2D, add
from keras.layers.advanced_activations import LeakyReLU
from keras.initializers import VarianceScaling
from keras.regularizers import l2
from keras.models import Model
from keras import backend as K
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
from IPython.display import SVG

Using TensorFlow backend.


In [3]:
def bn_conv2d(x, filter_size, kernel_size, padding_type, activation_type, strides=(1, 1)):
    """
    Batch Normalization and pre-activation followed by a convolution layer
    """    
    weight = 1e-4
    # Batch norm
    x = BatchNormalization(axis=-1, epsilon=1e-5, momentum=0.9)(x)
    # Pre-activation
    if activation_type == 'LeakyRelu':
        x = LeakyReLU(alpha=0.3)(x)
    else:
        x = Activation(activation_type)(x)
        
    # Convolution 2D
    x = Conv2D(filters=filter_size, kernel_size=kernel_size, strides=strides, kernel_regularizer=l2(weight),               
               kernel_initializer="he_normal", padding=padding_type, activation='linear', use_bias=False)(x)            
    return x

In [4]:
def residual_module(x, filter_size, stride_size, reduce=False):
    shortcut = x
    # First bn_conv2d block
    conv1 = bn_conv2d(x, filter_size=int(filter_size * 0.25), kernel_size=1, padding_type='same', 
                      activation_type='linear')
    # Second bn_conv2d block (this block also reduces spatial resolution)
    conv2 = bn_conv2d(conv1, filter_size=int(filter_size * 0.25), kernel_size=3, padding_type='same', 
                      activation_type='linear', strides=stride_size)
    # Third 
    conv3 = bn_conv2d(conv2, filter_size=filter_size, kernel_size=1, padding_type='same', activation_type='linear')
    
    if reduce is True:
        shortcut = bn_conv2d(x, filter_size, kernel_size=1, padding_type='same', activation_type='linear', strides=stride_size)        
    
    x = add([conv3, shortcut])
    
    return x            

In [5]:
def resnet_build(input_shape, num_classes, stage_list, filter_list):
    input_layer = Input(input_shape)
    # First convolution block to capture larger maps
    x = Conv2D(filters=filter_list[0], kernel_size=(5, 5), kernel_initializer="he_normal", 
               kernel_regularizer=l2(1e-4), padding='same', activation='linear')(input_layer)
    x = LeakyReLU(alpha=0.3)(x)
    x = BatchNormalization(axis=-1, epsilon=1e-5, momentum=0.9)(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    
    # loop over the number of stages and the number of filter to stack the residual modules
    
    for i in range(0, len(stage_list)):
        if (i == 0):
            stride_size = (1, 1)
        else:
            stride_size = (2, 2)
        
        x = residual_module(x, filter_list[i+1], stride_size=stride_size, reduce=True)
        
        for j in range(0, stage_list[i] - 1):
            x = residual_module(x, filter_list[i+1], stride_size=(1, 1))
    
    # head of the network
    x = BatchNormalization(axis=-1, epsilon=1e-5, momentum=0.9)(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = AveragePooling2D(pool_size=(8, 8))(x)
    x = Flatten()(x)
    x = Dense(units=num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=x)
    return model

In [6]:
input_shape = (64, 64, 3)
output_classes = 200
stage_list = (4, 6, 6)
filter_list = (64, 128, 256, 512)
model = resnet_build(input_shape, output_classes, stage_list=stage_list, filter_list=filter_list)

In [81]:
gt_label = 'horse'

In [86]:
pred_label = ['horse', 'dog', 'plant', 'machine']

In [87]:
if gt_label in pred_label[:5]:
    print('rank 5 accuracy')
if gt_label in pred_label[:1]:
    print('rank 1 accuracy')

rank 5 accuracy
rank 1 accuracy


In [89]:
from tqdm import tqdm
from keras.models import load_model

In [91]:
model = load_model(tqdm('C:/Users/PC/Downloads/tiny-imagenet-200/TinyImageNet_InceptionV4_Acc-0.46.hdf5'))


  0%|                                                                                           | 0/78 [00:00<?, ?it/s]

TypeError: expected str, bytes or os.PathLike object, not tqdm

In [92]:
5e-4

0.0005